In [1]:
# dataset
import os
from starry.utils.config import Configuration
from starry.utils.dataset_factory import loadDataset


DATA_DIR = os.environ.get('DATA_DIR')

config = Configuration.create('configs/evtopo-test.yaml')
train, val = loadDataset(config, data_dir=DATA_DIR, device='cpu')

it = iter(val)
tensors = next(it)

for k, v in tensors.items():
	print(f'{k}:\n', v[0].shape)



type:
 torch.Size([19])
staff:
 torch.Size([19])
feature:
 torch.Size([19, 15])
x:
 torch.Size([19])
y1:
 torch.Size([19])
y2:
 torch.Size([19])
matrixH:
 torch.Size([324])
tickDiff:
 torch.Size([19, 19])
maskT:
 torch.Size([19, 19])
tick:
 torch.Size([19])
division:
 torch.Size([19])
dots:
 torch.Size([19])
beam:
 torch.Size([19])
stemDirection:
 torch.Size([19])
grace:
 torch.Size([19])
timeWarped:
 torch.Size([19])
fullMeasure:
 torch.Size([19])
fake:
 torch.Size([19])


In [11]:
# tickDiff & maskT
tensors = next(it)

m2s = lambda m: '\n' + '\n'.join([',\t'.join(map(str, line)) for line in m.tolist()])

print('tickDiff:', m2s(tensors['tickDiff'][0]))
print('maskT:', m2s(tensors['maskT'][0]))


tickDiff: 
0.0,	0.0,	0.0,	-1920.0
0.0,	0.0,	0.0,	-1920.0
0.0,	0.0,	0.0,	-1920.0
1920.0,	1920.0,	1920.0,	0.0
maskT: 
False,	False,	False,	False
True,	False,	False,	False
True,	True,	False,	False
False,	True,	True,	False


In [3]:
# RectifySieveJointer
import torch
from starry.topology.models.rectifyJointer import RectifySieveJointer


model = RectifySieveJointer(1, 1)

with torch.no_grad():
	rec, j = model(tensors)

assert tensors['matrixH'][0].shape == j[0].shape

print('types:', tensors['type'].shape)
print('rec:', {k: v[0, 0] for k, v in rec.items()})
print('j:', len(j), j[0].shape)


types: torch.Size([4, 23])
rec: {'tick': tensor(-0.4209), 'division': tensor([0.1846, 0.1070, 0.1341, 0.1970, 0.0666, 0.1180, 0.1928]), 'dots': tensor([0.3660, 0.4169, 0.2171]), 'beam': tensor([0.3370, 0.3831, 0.0721, 0.2078]), 'stemDirection': tensor([0.5967, 0.1620, 0.2413]), 'grace': tensor(0.3963), 'timeWarped': tensor(0.5573), 'fullMeasure': tensor(0.6732), 'fake': tensor(0.4320)}
j: 4 torch.Size([484])


In [4]:
# RectifySieveJointerLoss
import torch
from starry.topology.models.rectifyJointer import RectifySieveJointerLoss


model = RectifySieveJointerLoss(n_trunk_layers=1, n_rectifier_layers=1)

with torch.no_grad():
	loss, metrics = model(tensors)

print('loss:', loss)
print('metrics:', {k: str(v) for k, v in metrics.items()})


loss: tensor(21.1035)
metrics: {'acc_topo': '0.0 (*1)', 'loss_topo': '1.1692346334457397 (*1)', 'error_tick': '987.3809814453125 (*92)', 'error_rel_tick': '209.44789123535156 (*168)', 'error_duration': '1918.701904296875 (*1)', 'acc_division': '0.33695653080940247 (*92)', 'acc_dots': '0.06521739065647125 (*92)', 'acc_beam': '0.28125 (*64)', 'acc_stemDirection': '0.078125 (*64)', 'acc_grace': '0.171875 (*64)', 'acc_timeWarped': '0.8913043737411499 (*92)', 'acc_fullMeasure': '0.8999999761581421 (*20)', 'acc_fake': '0.9347826242446899 (*92)'}


In [5]:
tensors = next(it)

with torch.no_grad():
	loss, metrics = model(tensors)

print('loss:', loss)
print('metrics:', {k: str(v) for k, v in metrics.items()})


loss: tensor(28.6344)
metrics: {'acc_topo': '0.0 (*1)', 'loss_topo': '1.928647518157959 (*1)', 'error_tick': '1066.08203125 (*64)', 'error_rel_tick': '467.9246826171875 (*92)', 'error_duration': '1918.568603515625 (*1)', 'acc_division': '0.234375 (*64)', 'acc_dots': '0.0 (*64)', 'acc_beam': '0.23076923191547394 (*52)', 'acc_stemDirection': '0.11538461595773697 (*52)', 'acc_grace': '0.11538461595773697 (*52)', 'acc_timeWarped': '0.90625 (*64)', 'acc_fullMeasure': '1.0 (*4)', 'acc_fake': '0.984375 (*64)'}


In [1]:
# RectifySieveJointer2
import os
import torch

from starry.utils.config import Configuration
from starry.utils.dataset_factory import loadDataset
from starry.utils.model_factory import loadModel


DATA_DIR = os.environ.get('DATA_DIR')

config = Configuration.create('configs/evtopo-test2.yaml', volatile=True)
train, val = loadDataset(config, data_dir=DATA_DIR, device='cpu')
model = loadModel(config['model'], postfix='Loss')

it = iter(val)

batch = next(it)
loss, metric = model(batch)

print('loss:', loss)
print('metric:', {key: metric[key].value for key in metric})


loss: tensor(25.9126, grad_fn=<AddBackward0>)
metric: {'acc_topo': 0.0, 'loss_topo': 1.5108212232589722, 'error_tick': 1079.7464599609375, 'error_rel_tick': 279.3047790527344, 'error_duration': 1919.5736083984375, 'acc_division': 0.19736842811107635, 'acc_dots': 0.3552631437778473, 'acc_beam': 0.3166666626930237, 'acc_stemDirection': 0.4000000059604645, 'acc_grace': 0.4833333194255829, 'acc_timeWarped': 0.07894736528396606, 'acc_fullMeasure': 0.0, 'acc_fake': 0.06578947603702545}
